In [ ]:
#%pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.8 MB/s eta 0:00:00a 0:00:01
Using cached click-8.1.8-py3-none-any.whl (98 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 3.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.1
    Uninstalling pydantic_core-2.33.1:
      Successfully uninstalled pydantic_core-2.33.1
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
  Attempting unin

In [7]:
from datetime import datetime
import gradio as gr
import os
from nClient import (
    create_socket, 
    connect_socket, 
    send_request,
    handle_response_content,
    save_response_content
)

def client_request(host, port, method, path, save_response, save_filename, headers, upload_file, manual_body, upload):
    host = host.strip() or "localhost"
    port = port.strip()
    port = int(port) if port.isdigit() else 80
    base_path = path.strip()

    # Remove protocol prefix
    for prefix in ["http://", "https://"]:
        if host.startswith(prefix):
            host = host[len(prefix):]
            break

    # Remove path suffix if present
    if "/" in host:
        host = host.split("/", 1)[0]

    # Check HTTP method
    method = method.upper().strip()
    if method not in ["GET", "HEAD", "POST", "PUT", "DELETE"]:
        return "Invalid method. Please enter GET, HEAD, POST, PUT, or DELETE."

    if method == 'GET':
        # Detect binary content by extension
        ext = path.split('.')[-1].lower() if '.' in path else ''
        is_binary = ext in ['png', 'jpg', 'jpeg', 'gif', 'mp3', 'wav', 'mp4', 'avi']


        # Construct and send request
        request = f"GET {path} HTTP/1.1\r\n"
        request += f"Host: {host}\r\n"
        request += "Connection: close\r\n"
        if is_binary:
            request += "Accept: */*\r\n"
        request += "\r\n"

        sock = create_socket()
        connect_socket(sock, host, port)
        response = send_request(sock, request, is_binary=is_binary)
        sock.close()

        if response:
            headers, content_type, content = handle_response_content(response, is_binary)
            if headers:
                print("\n=== Response Headers ===\n")
                print(headers)

                is_binary_content = content_type and any(t in content_type.lower()
                                                        for t in ['image/', 'audio/', 'video/', 'application/octet-stream'])

                if save_response and content:
                    if save_response_content(content, save_filename, is_binary_content):
                        print(f"\nContent saved to: {save_filename}")

                # Mostrar siempre la vista previa si no es binario
                if is_binary_content:
                    response = ("\n=== Response Headers ===\n") + headers
                if content and not is_binary_content:
                    print("\n=== Content Preview ===\n")
                    try:
                        print(content.decode() if isinstance(content, bytes) else content)
                    except:
                        print("(Binary content)")
        return response


    body = None  
    is_binary = False
    content_type = 'text/plain' 

    skip_file = False

    if method in ["POST","PUT"] and path.startswith("/resources"):
        parts = path.strip("/").split("/")
        if method == "POST":
            if len(parts) == 1:
                cat = input("Enter resource category: ").strip()
                path = f"/resources/{cat}"
            elif len(parts) == 2:
                pass
            else:
                return("Invalid POST path. Format: /resources/{category}")
                
        else:  # PUT
            if len(parts) < 3:
                if len(parts) == 1:
                    cat = input("Enter resource category: ").strip()
                    path = f"/resources/{cat}"
            else:
                return("Invalid PUT path. Format: /resources/{category}/{id}")
                
        body = manual_body
        print(body)
        content_type = "application/json"
        skip_file = True

    if method in ["POST","PUT"] and not skip_file:
        if upload:
            print("entra")
            try:
                print(upload_file)
                path = upload_file
                if isinstance(headers, str):
                    headers = headers.strip().splitlines()
                ext = upload_file.lower().split('.')[-1]
                print(ext)
                is_binary = ext in ['png','jpg','jpeg','gif','mp3','wav','mp4','avi']
                content_type = 'application/octet-stream'
                if ext in ['jpg','jpeg']:
                    content_type = 'image/jpeg'
                elif ext == 'png': content_type = 'image/png'
                elif ext == 'gif': content_type = 'image/gif'
                elif ext == 'mp3': content_type = 'audio/mpeg'
                elif ext == 'wav': content_type = 'audio/wav'
                elif ext in ['txt','html','css','json']:
                    content_type = 'text/plain'
                    is_binary = False
                mode = 'rb' if is_binary else 'r'
                with open(upload_file, mode) as f:
                    body = f.read()
                if is_binary:
                    print("binario")
                    boundary = "----WebKitFormBoundary" + datetime.now().strftime("%Y%m%d%H%M%S")
                    parts = [f"{method} {path} HTTP/1.1", f"Host: {host}", f"Content-Type: multipart/form-data; boundary={boundary}", f"Content-Length: {len(body)}"] + headers + [""]
                    request = "\r\n".join(parts).encode('utf-8') + b"\r\n" + body
                else:
                    parts = [f"{method} {path} HTTP/1.1", f"Host: {host}","Connection: close", f"Content-Type: {content_type}", f"Content-Length: {len(body)}"] + headers + [""]
                    request = "\r\n".join(parts) + "\r\n" + body
            except Exception as e:
                return(f"Error reading file: {e}")
                
        else:
            body = manual_body
            content_type = "application/json"

    # Construct and send request
    parts = [f"{method} {path} HTTP/1.1", f"Host: {host}", f"Content-Type: {content_type}"]
    if body is not None:
        parts.append(f"Content-Length: {len(body)}")
    parts.extend(headers)
    parts.append("")
    headers = "\r\n".join(parts)
    if body is not None:
        if isinstance(body, bytes):
            request = headers.encode('utf-8') + b"\r\n" + body
        else:
            request = headers + "\r\n" + body
    else:
        request = headers + "\r\n"

    sock = create_socket()
    connect_socket(sock, host, port)
    response = send_request(sock, request, is_binary=is_binary)
    sock.close()

    if response:
        print("\n=== Response ===\n")
        if upload:
            print("true")
        else:
            print("False")
        return response

    
    
"""
INTERFACE
"""
def update_ui(method, upload_file_checked):
    if method in ["POST", "PUT"]:
        return (
            gr.update(visible=True),   # save_file
            gr.update(visible=False),  # body_type
            gr.update(visible=False),  # manual_body hidden if file selected
            gr.update(visible=True),   # upload_file
            gr.update(visible=True),   # upload_file_checkbox
        )
    else:
        return (
            gr.update(visible=True),
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=False),
        )


def toggle_filename_input(save_file_checked):
    return gr.update(visible=save_file_checked)


def toggle_postPut_method(checked):
    if checked:
        return gr.update(visible=True), gr.update(visible=False)
    else:
        return gr.update(visible=False), gr.update(visible=True)


# Helper to extract file path
def get_file_path(file):
    return file.name if file else ""


with gr.Blocks() as iface:
    with gr.Row():
        host = gr.Textbox(label="Host", value="localhost")
        port = gr.Textbox(label="Port", value="8080")

    method = gr.Radio(
        choices=["GET", "POST", "PUT", "DELETE", "HEAD"],
        label="HTTP Method",
        value="GET"
    )

    path = gr.Textbox(label="Path", value="/")

    save_file = gr.Checkbox(label="Save file (for GET requests)", visible=True)
    file_name = gr.Textbox(label="Filename to save response", visible=False)
    save_file.change(fn=toggle_filename_input, inputs=save_file, outputs=file_name)

    headers = gr.Textbox(
        label="Custom Headers (one per line)",
        lines=3,
        placeholder="X-Custom: value"
    )

    upload_file_checkbox = gr.Checkbox(
        label="Upload File",
        value=False,
        visible=False
    )

    manual_body = gr.Textbox(
        label="Manual Body",
        lines=5,
        placeholder="Enter request body here...",
        visible=False
    )

    upload_file = gr.File(label="Upload File", type="filepath", visible=False)

    # Hidden textbox to hold the path of uploaded file
    file_path_box = gr.Textbox(visible=False)

    # Update visibility when checkbox is toggled
    upload_file_checkbox.change(
        fn=toggle_postPut_method,
        inputs=[upload_file_checkbox],
        outputs=[upload_file, manual_body]
    )

    # Update file path box when file is uploaded
    upload_file.change(
        fn=get_file_path,
        inputs=upload_file,
        outputs=file_path_box
    )

    # Update UI based on HTTP method
    method.change(
        update_ui,
        inputs=[method, upload_file_checkbox],
        outputs=[save_file, manual_body, upload_file, upload_file_checkbox, manual_body]
    )

    response = gr.Textbox(label="Response", lines=10)
    submit = gr.Button("Send Request")

    submit.click(
        fn=client_request,
        inputs=[
            host, port, method, path,
            save_file, file_name,
            headers, file_path_box, 
            manual_body, upload_file_checkbox
        ],
        outputs=response
    )


if __name__ == "__main__":
    iface.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


entra
/tmp/gradio/797c170ed9ce2f68e3b994e0aa4c7773b9b75b1569c213298259e1aeb09094e8/a.gif
gif
binario
Connected (plain) to localhost:8080

=== Response ===

true
Keyboard interruption in main thread... closing server.
